In [1]:
import re
import json
import string

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AffinityPropagation


In [2]:
def clean_string(string, regex=r'[^a-zA-Z\s0-9ÄÖÜäöü.?,;]+'):
    return re.sub(regex, '', string)

## read json data

In [3]:
with open('../data/dienstleistungen.json') as data_file:    
    data = json.load(data_file)

In [4]:
def parse_names(data):
    names = []
    for entry in data['data']:
        if entry['name'] == tuple():
            tmp = clean_string(entry['name'][0])
            names.append(tmp)
        else:
            tmp = clean_string(entry['name'])
            names.append(tmp)
    return names

In [5]:
names = parse_names(data)

## grouping based on field 'root_topic'

In [6]:
from collections import defaultdict

test = defaultdict(list)

for entry in data['data']:
    topic = entry['relation']['root_topic']
    test[topic].append(entry)

In [7]:
for topic, data in test.items():
    print('topic', topic)
    print()
    for services in data:
        print(services['name'])
    print()

topic 324813

Rundfunkbeitrag / Beitragspflicht, Befreiung vom Rundfunkbeitrag

topic 324823

Hundesteuer - Hund anmelden
Zweitwohnungsteuer - Festsetzung
Hundesteuer - Hund abmelden
Bescheinigung in Steuersachen - Ausstellung (ehemals steuerliche Unbedenklichkeitsbescheinigung)
Umsatzsteuer - Voranmeldungen, Jahreserklärungen
Lohnsteuerhilfeverein - Anerkennung
Kinderfreibeträge - Allgemeine Informationen
Grunderwerbsteuer - Festsetzung
Entlastungsbetrag für Alleinerziehende
Vergnügungsteuer - Festsetzung
Gewerbesteuer - Festsetzung
Körperschaftsteuer - Festsetzung
Steuerliche Abmeldung eines Unternehmens
Grundsteuer - Festsetzung
Gründungen - steuerliche Anmeldung von Unternehmen, Gesellschaften, Vereinen, Gewerben, Selbständigkeit oder land- und forstwirtschaftlicher Tätigkeit
Steueransprüche
Steueransprüche - Stundung
Arbeitnehmersparzulage - Gewährung
Wohnungsbauprämie - Festsetzung
Einkommensteuer - Festsetzung - Antragsveranlagung
Einkommensteuer - Festsetzung - Pflichtveranlagu

## clustering

In [8]:
punctuation_map = dict((ord(char), None) for char in string.punctuation)
stemmer = nltk.stem.snowball.GermanStemmer()

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize)

def get_clusters(sentences):
    tf_idf_matrix = vectorizer.fit_transform(sentences)
    similarity_matrix = (tf_idf_matrix * tf_idf_matrix.T).A
    affinity_propagation = AffinityPropagation(affinity="precomputed", damping=0.5)
    affinity_propagation.fit(similarity_matrix)

    labels = affinity_propagation.labels_
    cluster_centers = affinity_propagation.cluster_centers_indices_

    tagged_sentences = zip(sentences, labels)
    clusters = {}

    for sentence, cluster_id in tagged_sentences:
        clusters.setdefault(sentences[cluster_centers[cluster_id]], []).append(sentence)

    return clusters

def print_clusters(clusters):
    for cluster in clusters:
        print(cluster, ':')
        for element in clusters[cluster]:
            print('  - ', element)

## text clustering

In [9]:
print_clusters(get_clusters(names))

Abholung von sichergestellten Fahrzeugen :
  -  Vorführung von Fahrzeugen
  -  Besichtigung und Entnahmen von im Land Berlin sichergestellten Fahrzeugen durch den Berechtigten
  -  Abholung von sichergestellten Fahrzeugen
  -  Begutachtung von sichergestellten Fahrzeugen nur für bestellte Gutachter und Versicherungen
Straenrechtliche Sondernutzung Baustelleneinrichtung :
  -  Unternehmenskarte
  -  Straenrechtliche Sondernutzung Baustelleneinrichtung
  -  Straenrechtliche Sondernutzung Baugerüst
  -  Straenrechtliche Sondernutzung Kran, Schrägaufzug, Lift, Hebebühne
  -  Straenrechtliche Sondernutzung Baugrubenverbau, Bohr und Spundwand
  -  Straenrechtliche Sondernutzung Kellerlicht und Biereinwurfschacht
  -  Straenrechtliche Sondernutzung Erker, Balkon, Werbeschild an Gebäuden
  -  Straenrechtliche Sondernutzung Filmaufnahmen
  -  Straenrechtliche Sondernutzung  Aufstellen von Fahrradständern
  -  Straenrechtliche Sondernutzung Schrottcontainer
  -  Straenrechtliche Sondernutzung  A

## text clustering inside groups

In [10]:
for topic, data in test.items():
    sentences = []
    for services in data:
        #print(services['name'])
        sentences.append(services['name'])
    print_clusters(get_clusters(sentences))

/Users/sebastianhansmann/anaconda3/envs/tub/lib/python3.4/site-packages/sklearn/cluster/affinity_propagation_.py:133: RuntimeWarning: invalid value encountered in subtract
  tmp -= np.sum(tmp, axis=0)


TypeError: 'NoneType' object is not subscriptable